This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [5]:
# First, import the relevant modules
import os
import requests
import json
from dotenv import load_dotenv

load_dotenv()
API_KEY=os.getenv('NASDAQ_API_KEY')

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [16]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
start_date='2017-01-01'
end_date = '2017-12-31'
base_url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json"
params={'api_key':API_KEY,'start_date':start_date,'end_date':end_date,'limit':1}
response=requests.get(base_url,params=params)
print(response.content)

b'{"dataset_data":{"limit":1,"transform":null,"column_index":null,"column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"start_date":"2017-01-01","end_date":"2017-12-31","frequency":"daily","data":[["2017-12-29",51.76,51.94,51.45,51.76,null,34640.0,1792304.0,null,null,null]],"collapse":null,"order":null}}'


In [10]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

### Solution
1. The start and end dates, api key parameters, url variables are defined.
2. The response is converted to a python dictionary.
3. Lists 'column_names' and 'data' are used to store the column names and the values of the fields respectively

In [18]:
start_date='2017-01-01'
end_date = '2017-12-31'
base_url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json"
params={'api_key':API_KEY,'start_date':start_date,'end_date':end_date}
response=requests.get(base_url,params=params)
data=json.loads(response.content)
column_names=data['dataset_data']['column_names']
data=data['dataset_data']['data']

In [26]:
#### List of column names in the response
for col in column_names:
    print(col)

Date
Open
High
Low
Close
Change
Traded Volume
Turnover
Last Price of the Day
Daily Traded Units
Daily Turnover



4. Lists are created for all records in the api response, for the following column values
    1. open_prices- list of 'Open' price per day 
    2. day_diff - list of difference between 'High' and 'Low' per day
    2. closing_price-  list of closing price per day
    3. traded_volume- list of 'Traded Volume' per day

    

In [34]:
    open_prices=[]
    day_diff=[]
    closing_price=[]
    traded_volume=[]

6. The index values for the 'High', 'Low','Close' and 'Traded Volume' are assigned to variables. These index values are used to loop through the data list to apppend values to the different field lists

In [32]:
    for i in range(len(column_names)):
        if column_names[i] == 'Open':
            idx_open = i
        if column_names[i] == 'Close':
            idx_close = i
        if column_names[i] == 'Traded Volume':
            idx_tradedv = i
        if column_names[i] == 'High':
            idx_high = i
        if column_names[i] == 'Low':
            idx_low = i

7. The different values are calculated from the created lists .

In [33]:
    for data_row in data:
        if data_row[idx_open]:
            open_prices.append(data_row[idx_open])
        if data_row[idx_high]-data_row[idx_low]:
            day_diff.append(data_row[idx_high]-data_row[idx_low])
        if data_row[idx_close]:
            closing_price.append(data_row[idx_close])
        if data_row[idx_tradedv]:
            traded_volume.append(data_row[idx_tradedv])

    traded_volume.sort()
    if len(traded_volume) % 2:

        median = traded_volume[len(traded_volume) // 2]
    else:
        median = (traded_volume[(len(traded_volume) // 2) - 1] + traded_volume[(len(traded_volume) // 2)]) / 2


    print(f"Lowest opening price : {round(min(open_prices),2)}")
    print(f"Highest opening price :{round(max(open_prices),2)}")
    print(f"largest change in any one day (based on High and Low price) : {round(max(day_diff),2)}")
    print(f"argest change between any two days (based on Closing Price) : {round(max(closing_price)-min(closing_price),2)}")
    print(f"Average daily trading volume during this year : {round(sum(traded_volume)/len(traded_volume),2)}")
    print(f"median trading volume during this year : {round(median,2)}")

Lowest opening price : 34.0
Highest opening price :53.11
largest change in any one day (based on High and Low price) : 2.81
argest change between any two days (based on Closing Price) : 19.03
Average daily trading volume during this year : 89124.34
median trading volume during this year : 76286.0
